In [94]:
import pandas as pd

import decimal as dcm
def usual_rounding (x):
    #need this because python rounding use banker method which is stupid in our education
    return float(dcm.Decimal(str(x)).quantize(dcm.Decimal('1.00'), rounding=dcm.ROUND_HALF_UP))

# Các quy tắc điểm chính

## Đổi điểm theo bách phân vị

In [95]:
def point_converter (case_table, initial_option, initial_score, target_option):
    initial_df = case_table.iloc[:,initial_option]

    ranking_index = -1
    for i in range(len(initial_df.index)):
        if initial_score <= initial_df.iloc[i]:
            ranking_index = i

    if ranking_index >= len(initial_df.index) - 1: 
        return None
    else:
        target_df = case_table.iloc[:,target_option]

        initial_upper = float(initial_df.iloc[ranking_index])
        initial_lower = float(initial_df.iloc[ranking_index+1])
        target_upper = float(target_df.iloc[ranking_index])
        target_lower = float(target_df.iloc[ranking_index+1])

        target_score = target_lower + (target_upper - target_lower)/(initial_upper - initial_lower) * (initial_score - initial_lower)

        return usual_rounding(target_score)


In [96]:
def conversion_result_table (case_table, initial_option, initial_score):
    option_names = list(case_table.columns)

    # Collect conversion results
    row = {}

    for idx, name in enumerate(option_names):
        temp = point_converter(case_table, initial_option, initial_score, idx)
        if temp is None:
            print("Không đủ điểm sàn. Chúc bạn may mắn năm sau")
            break
        row[name] = temp

    if row:  # Only append if row is not empty
        result_table = pd.DataFrame([row])
        print("Bảng quy đổi các đầu điểm: ")
        display(result_table.style.hide(axis="index").format(precision=2))

## Điểm ưu tiên (giải quốc gia, đối tượng ưu tiên, khu vực ưu tiên)

### Thang điểm 30

In [97]:
def priority_score_in_30 (original_score):
    print("\nNhập điểm ưu tiên giải cấp QG nếu có (Nhất: 3; Nhì: 2; Ba: 1.5; KK: 1): "); prize_priority_score = float(input())
    print("Nhập tổng điểm ưu tiên vùng và đối tương (KV1: 0.75; KV2NT: 0.5; KV2: 0.25; KV3: 0; Nhóm UT1: 2; Nhóm UT2: 1): "); region_prority_score = float(input())

    current_score = original_score + prize_priority_score
    
    if current_score >= 30: final_score = 30
    else:
        if current_score <= 22.5: region_priority_score_normalize = region_prority_score
        else:
            region_priority_score_normalize = (30 - current_score)/7.5 * region_prority_score
        
        final_score = current_score + region_priority_score_normalize
        if final_score >= 30: final_score = 30

    return final_score

### Thang điểm 40

In [98]:
def priority_score_in_40 (original_score):
    print("\nNhập điểm ưu tiên giải cấp QG nếu có (Nhất: 3; Nhì: 2; Ba: 1.5; KK: 1): "); prize_priority_score = float(input())
    print("Nhập tổng điểm ưu tiên vùng và đối tương (KV1: 0.75; KV2NT: 0.5; KV2: 0.25; KV3: 0; Nhóm UT1: 2; Nhóm UT2: 1): "); region_prority_score = float(input())

    current_score = original_score + prize_priority_score/3.0 * 4
    
    if current_score >= 40: final_score = 40
    else:
        if current_score <= 30: region_priority_score_normalize = region_prority_score/3.0 * 4
        else:
            region_priority_score_normalize = (40 - current_score)/10.0 * (region_prority_score/3.0 * 4)
        
        final_score = current_score + region_priority_score_normalize
        if final_score >= 40: final_score = 40

    return final_score

## Điểm quy đổi các CCNN quốc tế / giải HSGQG / ĐGNL Quốc tế

In [99]:
ielts_dict = {8.0: 10.0, 7.5: 9.5, 7.0: 9.0, 6.5: 8.5}
toefl_dict = {110: 10.0, 102: 9.5, 93: 9.0, 79: 8.5}
cambridge_dict = {200: 10.0, 192: 9.5, 184: 9.0, 180: 8.5}
jlpt_dict = {"N1": 10.0, "N2": 9.5, "N3": 9.0}
eju_dict = {320: 10.0, 270: 9.5, 220: 9.0}
hsk_dict = {"HSK6": 10.0, "HSK5": 9.5, "HSK4": 9.0}
delf_dict = {"C2": 10.0, "C1": 9.5, "B2": 9.0}
qgia_dict = {"Nhất": 10.0, "Nhì": 9.5, "Ba": 9.0}

sat_dict = {1550: 20.00, 1530: 19.75, 1500: 19.50, 1480: 19.00, 1430: 18.50, 1390: 18.25, 1380: 18.00}
act_dict = {36: 20.00, 35: 19.75, 34: 19.50, 33: 19.00, 32: 18.50, 31: 18.25, 30: 18.00}
alvl_dict = {"A*": 10.0, "A": 9.0, "B": 8.0, "C": 7.5, "D": 7.0, "E": 6.5}

In [100]:
def finding_type_one (case_table, initial_score):
    final_score = 0.0
    for keys in case_table:
        if initial_score >= keys: 
            final_score = case_table[keys]
            break

    return final_score

def finding_type_two (case_table, initial_score):
    final_score = 0.0
    for keys in case_table:
        if initial_score == keys: 
            final_score = case_table[keys]
            break

    return final_score

In [101]:
def ccnnqt_conversion ():
    print("""Chọn phương thức quy đổi CCNNQT (
          1: IELTS <thang 9>
          2: TOEFL <thang 120> 
          3: Cambridge <thang 230> 
          4: JLPT <loại N1, N2, N3, ...>
          5: EJU <thang 400> 
          6: HSK <loại HSK6, HSK5, HSK4, ...>
          7: DELF <loại C2, C1, B2, ...>
          8: HSGQG <loại Nhất, Nhì, Ba, ...> ): """); type = int(input())
    print("\n")

    if type in [1,2,3,5]:
        if type == 1: case_table = ielts_dict
        elif type == 2: case_table = toefl_dict
        elif type == 3: case_table = cambridge_dict
        else: case_table = eju_dict
        
        print("Nhập số điểm ban đầu: "); score = int(input())
        final_score = finding_type_one(case_table, score)
        print(f"Điểm quy đổi sang thang 10 là: {final_score}")

    elif type in [4,6,7,8]:
        if type == 4: case_table = jlpt_dict
        elif type == 6: case_table = hsk_dict
        elif type == 7: case_table = delf_dict
        else: case_table = qgia_dict

        print("Nhập loại điểm ban đầu: "); score = str(input())
        final_score = finding_type_two(case_table, score)
        print(f"Điểm quy đổi sang thang 10 là: {final_score}")

    else: final_score = 0

    return final_score


## Điểm quy đổi các kì thi Đánh giá Năng lực quốc tế

In [102]:
def dgnlqt_conversion ():
    print("""\nChọn phương thức ĐGNLQT (
          1: SAT <thang 1600>
          2: ACT <thang 36>
          3: A-Level <loại A*,A,B,C,D,E> ): """); type = int(input())

    if type in [1,2]:
        if type == 1: case_table = sat_dict
        else: case_table = act_dict

        print("Nhập số điểm ban đầu: "); score = int(input())
        final_score = finding_type_one(case_table, score)
        print(f"Điểm quy đổi sang thang 20 là: {final_score}")

    elif type == 3:
        case_table = alvl_dict
        
        print("Nhập loại điểm ban đầu: "); score = str(input())
        final_score = finding_type_two(case_table, score)
        print(f"Điểm quy đổi sang thang 10 là: {final_score}")

    else: final_score = 0

    return final_score


# Dữ liệu bách phân vị từ trường

## Nhóm 1: Chương trình Tiêu chuẩn (không bao gồm Chương trình Thương mại)

Lưu ý: Với các bạn THPTQG thi các khối A01, D01, D02, D03, D04, D06, D07, vui lòng cộng 1 điểm để đưa điểm về tổ hợp A00, rồi mới quy đổi

Ngược lại: Khi trường công bố điểm chuẩn THPTQG cho khối A00, các bạn trừ đi 1 điểm sẽ ra điểm chuẩn cho các khối còn lại. Ví dụ: 20.5 điểm A00 <-> 19.5 điểm D01

In [103]:
thpt_list_1 = [30, 28, 27.3, 26.75, 26, 25.25, 24]
hsgqg_list_1 = [30, 28.41, 28.02, 27.5, 26.62, 26.11, 25.35]
hsgtp_list_1 = [30, 28.62, 28.38, 28.12, 27.81, 27.5, 27]
hsa_list_1 = [30, 28.2, 28, 27.8, 27.6, 27.3, 27]
vact_list_1 = [30, 28.51, 28.4, 28.29, 28.24, 28.16, 27]

data_1 = {"THPT(A00)": thpt_list_1, "HSGQG": hsgqg_list_1, "HSGTP": hsgtp_list_1, "HSA": hsa_list_1, "V-ACT": vact_list_1}
table_1 = pd.DataFrame(data_1)
display(table_1.style.hide(axis="index").format(precision=2))

THPT(A00),HSGQG,HSGTP,HSA,V-ACT
30.00,30.00,30.00,30.00,30.00
28.00,28.41,28.62,28.20,28.51
27.30,28.02,28.38,28.00,28.40
26.75,27.50,28.12,27.80,28.29
26.00,26.62,27.81,27.60,28.24
25.25,26.11,27.50,27.30,28.16
24.00,25.35,27.00,27.00,27.00


## Nhóm 2: Chương trình Chất lượng cao và Chương trình tiên tiến (không bao gồm Chương trình Thương mại)

Lưu ý: D01 là tổ hợp gốc ở trường hợp này.

In [104]:
thpt_ccnnqt_list_2 = [30, 28.1, 27.5, 26.8, 26.2, 24.5, 23]
hsgqg_ccnnqt_list_2 = [30, 27.85, 27.5, 27.22, 27.05, 26.81, 25.5]
hsgtp_ccnnqt_list_2 = [30, 28.68, 28.57, 28.35, 27.89, 27.25, 25.5]
dgnlqt_ccnnqt_list_2 = [30, 29.5, 29, 28.75, 28.5, 27.75, 26.5]

data_2 = {"THPT_CCNNQT": thpt_ccnnqt_list_2, 
          "HSGQG_CCNNQT": hsgqg_ccnnqt_list_2, 
          "HSGTP_CCNNQT": hsgtp_ccnnqt_list_2, 
          "DGNLQT_CCNNQT": dgnlqt_ccnnqt_list_2}

table_2 = pd.DataFrame(data_2)
display(table_2.style.hide(axis="index").format(precision=2))

THPT_CCNNQT,HSGQG_CCNNQT,HSGTP_CCNNQT,DGNLQT_CCNNQT
30.00,30.00,30.00,30.00
28.10,27.85,28.68,29.50
27.50,27.50,28.57,29.00
26.80,27.22,28.35,28.75
26.20,27.05,27.89,28.50
24.50,26.81,27.25,27.75
23.00,25.50,25.50,26.50


## Nhóm 3: Chương trình Định hướng nghề nghiệp & Phát triển quốc tế

Lưu ý: Với các bạn THPTQG thi các khối A01, D01, D06, D07, vui lòng cộng 1 điểm để đưa điểm về tổ hợp A00, rồi mới quy đổi

Ngược lại: Khi trường công bố điểm chuẩn THPTQG cho khối A00, các bạn trừ đi 1 điểm sẽ ra điểm chuẩn cho các khối còn lại. Ví dụ: 20.5 điểm A00 <-> 19.5 điểm D01

In [105]:
thpt_ccnnqt_list_3 = [30, 28.1, 27.5, 26.8, 26.2, 24.5, 23]
hsgqg_ccnnqt_list_3 = [30, 27.85, 27.5, 27.22, 27.05, 26.81, 26.53]
hsgtp_ccnnqt_list_3 = [30, 28.68, 28.57, 28.35, 27.89, 27.25, 25.5]
dgnlqt_ccnnqt_list_3 = [30, 29.5, 29, 28.75, 28.5, 27.75, 26.5]
thpt_a00_list_3 = [30, 28, 27.3, 26.75, 26, 25.25, 24]
hsgqg_thpt3_list_3 = [30, 28.41, 28.02, 27.5, 26.62, 26.11, 25.35]
hsgtp_thpt3_list_3 = [30, 28.62, 28.38, 28.12, 27.81, 27.5, 27]
dgnl_hsa_list_3 = [30, 28.2, 28, 27.8, 27.6, 27.3, 27]
dgnl_vact_list_3 = [30, 28.51, 28.4, 28.29, 28.24, 28.16, 27]

data_3 = {
    "THPT(A00)": thpt_a00_list_3,
    "HSGQG_THPT3": hsgqg_thpt3_list_3,
    "HSGTP_THPT3": hsgtp_thpt3_list_3,
    "DGNL_HSA": dgnl_hsa_list_3,
    "DGNL_VACT": dgnl_vact_list_3,
    "THPT_CCNNQT": thpt_ccnnqt_list_3,
    "HSGQG_CCNNQT": hsgqg_ccnnqt_list_3,
    "HSGTP_CCNNQT": hsgtp_ccnnqt_list_3,
    "DGNLQT_CCNNQT": dgnlqt_ccnnqt_list_3
}

table_3 = pd.DataFrame(data_3)
display(table_3.style.hide(axis="index").format(precision=2))

THPT(A00),HSGQG_THPT3,HSGTP_THPT3,DGNL_HSA,DGNL_VACT,THPT_CCNNQT,HSGQG_CCNNQT,HSGTP_CCNNQT,DGNLQT_CCNNQT
30.00,30.00,30.00,30.00,30.00,30.00,30.00,30.00,30.00
28.00,28.41,28.62,28.20,28.51,28.10,27.85,28.68,29.50
27.30,28.02,28.38,28.00,28.40,27.50,27.50,28.57,29.00
26.75,27.50,28.12,27.80,28.29,26.80,27.22,28.35,28.75
26.00,26.62,27.81,27.60,28.24,26.20,27.05,27.89,28.50
25.25,26.11,27.50,27.30,28.16,24.50,26.81,27.25,27.75
24.00,25.35,27.00,27.00,27.00,23.00,26.53,25.50,26.50


## Nhóm 4: Chương trình Khoa học máy tính & Dữ liệu trong Kinh tế & Kinh doanh

Lưu ý: Không có độ lệch điểm giữa các tổ hợp THPTQG khi xét trường hợp này. 20,5 điểm A00 <-> 20,5 điểm D01

In [106]:
thpt_a00_list_4 = [40, 37.33, 34.8, 32]
hsgqg_thpt3_list_4 = [40, 39.24, 37.4, 33.52]
hsgtp_thpt3_list_4 = [40, 38.54, 37, 36]
dgnl_hsa_list_4 = [40, 37.25, 36.29, 36]
dgnl_vact_list_4 = [40, 37.86, 37.55, 36]
thpt_ccnnqt_list_4 = [40, 37.47, 35.07, 31]
hsgqg_ccnnqt_list_4 = [40, 39.24, 37.61, 34.42]
hsgtp_ccnnqt_list_4 = [40, 38.07, 36.37, 34]
dgnlqt_ccnnqt_list_4 = [40, 39, 37.33, 36]

data_4 = {
    "THPT(A00)": thpt_a00_list_4,
    "HSGQG_THPT3": hsgqg_thpt3_list_4,
    "HSGTP_THPT3": hsgtp_thpt3_list_4,
    "DGNL_HSA": dgnl_hsa_list_4,
    "DGNL_VACT": dgnl_vact_list_4,
    "THPT_CCNNQT": thpt_ccnnqt_list_4,
    "HSGQG_CCNNQT": hsgqg_ccnnqt_list_4,
    "HSGTP_CCNNQT": hsgtp_ccnnqt_list_4,
    "DGNLQT_CCNNQT": dgnlqt_ccnnqt_list_4
}

table_4 = pd.DataFrame(data_4)
display(table_4.style.hide(axis="index").format(precision=2))

THPT(A00),HSGQG_THPT3,HSGTP_THPT3,DGNL_HSA,DGNL_VACT,THPT_CCNNQT,HSGQG_CCNNQT,HSGTP_CCNNQT,DGNLQT_CCNNQT
40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00
37.33,39.24,38.54,37.25,37.86,37.47,39.24,38.07,39.00
34.80,37.40,37.00,36.29,37.55,35.07,37.61,36.37,37.33
32.00,33.52,36.00,36.00,36.00,31.00,34.42,34.00,36.00


## Nhóm 5: Chương trình tiêu chuẩn Ngôn ngữ thương mại

Lưu ý: Ngành chỉ tuyển các tổ hợp trọng tâm Ngoại ngữ như D01 (Tiếng Anh), D03 (Tiếng Pháp), D04 (Tiếng Trung), D06 (Tiếng Nhật). Không có độ lệch giữa các tổ hợp trên.

In [107]:
thpt_list_5 = [40, 37.88, 36.3, 34.7, 30]
hsgqg_list_5 = [40, 38.66, 37.62, 35.87, 33.68]
hsgtp_list_5 = [40, 37.69, 37.2, 36.33, 36]
hsa_list_5 = [40, 37.53, 37.29, 36.56, 36]

data_5 = {"THPT(D01)": thpt_list_5, 
          "HSGQG": hsgqg_list_5, 
          "HSGTP": hsgtp_list_5, 
          "HSA": hsa_list_5}

table_5 = pd.DataFrame(data_5)
display(table_5.style.hide(axis="index").format(precision=2))

THPT(D01),HSGQG,HSGTP,HSA
40.00,40.00,40.00,40.00
37.88,38.66,37.69,37.53
36.30,37.62,37.20,37.29
34.70,35.87,36.33,36.56
30.00,33.68,36.00,36.00


## Nhóm 6: Chương trình Chất lượng cao Ngôn ngữ thương mại

Lưu ý: Ngành chỉ tuyển các tổ hợp trọng tâm Ngoại ngữ như D01 (Tiếng Anh), D03 (Tiếng Pháp), D04 (Tiếng Trung), D06 (Tiếng Nhật). Không có độ lệch giữa các tổ hợp trên.

In [108]:
thpt_ccnnqt_list_6 = [40, 37.47, 35.3, 33.3, 30]
hsgqg_ccnnqt_list_6 = [40, 38.17, 37.64, 35.86, 34.07]
hsgtp_ccnnqt_list_6 = [40, 36.13, 35.67, 35.02, 33.41]
dqnlqt_ccnnqt_list_6 = [40, 38.07, 37.5, 36.62, 35.5]

data_6 = {"THPT(D01)_CCNNQT": thpt_ccnnqt_list_6, 
          "HSGQG_CCNNQT": hsgqg_ccnnqt_list_6, 
          "HSGTP_CCNNQT": hsgtp_ccnnqt_list_6, 
          "DGNLQT_CCNNQT": dqnlqt_ccnnqt_list_6}

table_6 = pd.DataFrame(data_6)
display(table_6.style.hide(axis="index").format(precision=2))

THPT(D01)_CCNNQT,HSGQG_CCNNQT,HSGTP_CCNNQT,DGNLQT_CCNNQT
40.00,40.00,40.00,40.00
37.47,38.17,36.13,38.07
35.30,37.64,35.67,37.50
33.30,35.86,35.02,36.62
30.00,34.07,33.41,35.50


# Các quy đổi theo quy chế

## Điểm thi THPTQG

### Nhóm ngành thường

In [109]:
def thpt_normal_conversion (case):
    tmp = 0

    if case in [1, 3]: #nhóm ngành tiêu chuẩn + dhnn&ptqt; a00 gốc
        print("Nhập lần lượt điểm 3 môn THPTQG (nếu bạn thi tổ hợp khác A00, cộng 1 điểm vào 1 môn bất kì để đổi điểm về A00):")
        print("Môn 1: "); subject_1 = float(input())
        print("Môn 2: "); subject_2 = float(input())
        print("Môn 3: "); subject_3 = float(input())

        tmp = subject_1 + subject_2 + subject_3
        return priority_score_in_30(tmp)

    if case in [4, 5]: #khmt&dl trong kt&kd; toán * 2 ; ko cần hiệu chỉnh - tiêu chuẩn thương mại; duy nhất khối d; anh * 2
        if case == 4: 
            print("Nhập lần lượt điểm Toán và 2 môn THPTQG còn lại (tổ hợp chọn theo quy chế):")
            print("Môn Toán: "); subject_1 = float(input())
        else:
            print("Nhập lần lượt điểm Ngoại ngữ và 2 môn THPTQG còn lại (tổ hợp chọn theo quy chế):")
            print("Môn Ngoại ngữ: "); subject_1 = float(input())
        print("Môn còn lại 1: "); subject_2 = float(input())
        print("Môn còn lại 2: "); subject_3 = float(input())

        tmp = subject_1 * 2 + subject_2 + subject_3
        return priority_score_in_40(tmp)
    

### Kết hợp Chứng chi ngoại ngữ

In [110]:
def thpt_certificate_conversion (case):
    tmp = 0

    if case in [2, 3]: #nhóm ngành clc/cttt + dhnn&ptqt; a00 gốc
        print("Nhập lần lượt các đầu điểm: (nếu bạn thi tổ hợp khác A00, cộng 1 điểm vào môn THPTQG bất kì như Toán/Lý/Hóa/Văn để đổi điểm về A00):")
        print("Môn Toán: "); subject_1 = float(input())
        print("Môn Lý/Hóa/Văn: "); subject_2 = float(input())
        print("Điểm quy đổi ngoại ngữ: "); subject_3 = ccnnqt_conversion()

        tmp = subject_1 + subject_2 + subject_3
        return priority_score_in_30(tmp)

    if case in [4, 6]: #khmt&dl trong kt&kd; toán * 2 ; ko cần hiệu chỉnh - tiêu chuẩn thương mại; duy nhất khối d; anh * 2
        scalar_1 = 2; scalar_2 = scalar_3 = 1
        print("Nhập lần lượt các đầu điểm:")
        print("Môn Toán: "); subject_1 = float(input())
        if case == 4: 
            print("Môn Lý/Hóa/Văn: "); subject_2 = float(input())
            print("Điểm quy đổi ngoại ngữ: "); subject_3 = cnnqt_conversion()
            tmp = subject_1 * 2 + subject_2 + subject_3
        else: 
            print("Môn Văn: "); subject_2 = float(input())
            print("Điểm quy đổi ngoại ngữ: "); subject_3 = cnnqt_conversion()
            tmp = subject_1 + subject_2 + subject_3 * 2

        return priority_score_in_40(tmp)
    

## Điểm thi DGNL HSA từ thang 150 sang thang 30

In [111]:
def hsa_normal_conversion_to_30 ():
    print("Nhập điểm HSA (thang 150): "); hsa_score = int(input())

    starting_score = 27
    hsa_score_normalize = (hsa_score - 100)/50.0 * 3

    tmp = starting_score + hsa_score_normalize

    return priority_score_in_30(tmp)

## Điểm thi DGNL V-ACT từ thang 150 sang thang 30

In [112]:
def vact_normal_conversion_to_30 ():
    print("Nhập điểm V-ACT (thang 1200): "); vact_score = int(input())
    
    starting_score = 27
    vact_score_normalize = (vact_score - 850)/350.0 * 3

    tmp = starting_score + vact_score_normalize
    
    return priority_score_in_30(tmp)

## Điểm thi Đánh giá năng lực quốc tế + Chứng chỉ Ngoại ngữ Quốc tế

# Menu

In [113]:
def option_1_menu():
    case_table = table_1

    print("""Chọn phương thức xét tuyển (
          1: Kết quả THPTQG
          2: Đối tượng HSG Quốc gia
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên
          4: Kết quả DGNL HSA
          5: Kết quả DGNL V-ACT) """); option = int(input())
    print("\n")
    
    if option == 1: 
        initial_score = usual_rounding(thpt_normal_conversion(case = 1))
        print(f"Điểm THPTQG quy đổi sang khối A00 thang 30 là: {initial_score}\n")
    elif option == 2: print("PT2")
    elif option == 3: print("PT3")
    elif option == 4: 
        initial_score = usual_rounding(hsa_normal_conversion_to_30())
        print(f"Điểm HSA quy đổi sang thang 30 là: {initial_score}\n")
    elif option == 5:
        initial_score = usual_rounding(vact_normal_conversion_to_30())
        print(f"Điểm V-ACT quy đổi sang thang 30 là: {initial_score}\n")
    else: print("Đầu vào không hợp lệ\n")

    conversion_result_table (case_table, option-1, initial_score)

    return

In [114]:
def option_2_menu():
    case_table = table_2

    print("""Chọn phương thức xét tuyển (
          1: Kết quả THPTQG + CCNNQT
          2: Đối tượng HSG Quốc gia + CCNNQT
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên + CCNNQT
          4: Kết quả DGNL Quôc tế + CCNNQT ): """); option = int(input())
    print("\n")
    
    if option == 1:
        initial_score = usual_rounding(thpt_certificate_conversion(case = 2))
        print(f"Điểm THPTQG + CCNNQT quy đổi sang thang 30 là: {initial_score}\n")
    elif option == 2: print("PT2")
    elif option == 3: print("PT3")
    elif option == 4: print("PT4")
    else: print("Đầu vào không hợp lệ\n")

    conversion_result_table (case_table, option-1, initial_score)

    return

In [115]:
def option_3_menu():
    case_table = table_3

    print("""Chọn phương thức xét tuyển (
          1: Kết quả THPTQG
          2: Đối tượng HSG Quốc gia
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên
          4: Kết quả DGNL HSA
          5: Kết quả DGNL V-ACT
          6: Kết quả THPTQG + CCNNQT
          7: Đối tượng HSG Quốc gia + CCNNQT
          8: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên + CCNNQT
          9: Kết quả DGNL Quôc tế + CCNNQT ): """); option = int(input())
    print("\n")
    
    if option == 1:
        initial_score = usual_rounding(thpt_normal_conversion(case = 3))
        print(f"Điểm THPTQG quy đổi sang khối A00 thang 30 là: {initial_score}\n")
    elif option == 2: print("PT2")
    elif option == 3: print("PT3")
    elif option == 4:
        initial_score = usual_rounding(hsa_normal_conversion_to_30())
        print(f"Điểm HSA quy đổi sang thang 30 là: {initial_score}\n")
    elif option == 5:
        initial_score = usual_rounding(vact_normal_conversion_to_30())
        print(f"Điểm V-ACT quy đổi sang thang 30 là: {initial_score}\n")
    elif option == 6:
        initial_score = usual_rounding(thpt_certificate_conversion(case = 2))
        print(f"Điểm THPTQG + CCNNQT quy đổi sang thang 30 là: {initial_score}\n")
    elif option == 7: print("PT7")
    elif option == 8: print("PT8")
    elif option == 9: print("PT9")
    else: print("Đầu vào không hợp lệ\n")

    conversion_result_table (case_table, option-1, initial_score)

    return

In [116]:
def option_4_menu():
    case_table = table_4

    print("""Chọn phương thức xét tuyển (
          1: Kết quả THPTQG
          2: Đối tượng HSG Quốc gia
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên
          4: Kết quả DGNL HSA
          5: Kết quả DGNL V-ACT
          6: Kết quả THPTQG + CCNNQT
          7: Đối tượng HSG Quốc gia + CCNNQT
          8: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên + CCNNQT
          9: Kết quả DGNL Quôc tế + CCNNQT ): """); option = int(input())
    print("\n")
    
    if option == 1: 
        initial_score = usual_rounding(thpt_normal_conversion(case = 4))
        print(f"Điểm THPTQG quy đổi sang thang 40 là: {initial_score}\n")
    elif option == 2: print("PT2")
    elif option == 3: print("PT3")
    elif option == 4:
        initial_score = usual_rounding(hsa_normal_conversion_to_30()/3.0 * 4)
        print(f"Điểm HSA quy đổi sang thang 40 là: {initial_score}\n")
    elif option == 5:
        initial_score = usual_rounding(vact_normal_conversion_to_30()/3.0 * 4)
        print(f"Điểm V-ACT quy đổi sang thang 40 là: {initial_score}\n")
    elif option == 6:
        initial_score = usual_rounding(thpt_certificate_conversion(case = 2))
        print(f"Điểm THPTQG + CCNNQT quy đổi sang thang 40 là: {initial_score}\n")
    elif option == 7: print("PT7")
    elif option == 8: print("PT8")
    elif option == 9: print("PT9")
    else: print("Đầu vào không hợp lệ\n")

    conversion_result_table (case_table, option-1, initial_score)

    return

In [117]:
def option_5_menu():
    case_table = table_5

    print("""Chọn phương thức xét tuyển (
          1: Kết quả THPTQG
          2: Đối tượng HSG Quốc gia
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên
          4: Kết quả DGNL HSA ): """); option = int(input())
    print("\n")
    
    if option == 1: 
        initial_score = usual_rounding(thpt_normal_conversion(case = 5))
        print(f"Điểm THPTQG quy đổi sang thang 40 là: {initial_score}\n")
    elif option == 2: print("PT2")
    elif option == 3: print("PT3")
    elif option == 4: 
        initial_score = usual_rounding(hsa_normal_conversion_to_30()/3.0 * 4)
        print(f"Điểm HSA quy đổi sang thang 40 là: {initial_score}\n")
    else: print("Đầu vào không hợp lệ\n")

    conversion_result_table (case_table, option-1, initial_score)

    return

In [118]:
def option_6_menu():
    case_table = table_6

    print("""Chọn phương thức xét tuyển (
          1: Kết quả THPTQG (khối D) + CCNNQT
          2: Đối tượng HSG Quốc gia + CCNNQT
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên + CCNNQT
          4: Kết quả DGNL Quôc tế + CCNNQT ): """); option = int(input())
    print("\n")
    
    if option == 1:
        initial_score = usual_rounding(thpt_certificate_conversion(case = 2))
        print(f"Điểm THPTQG + CCNNQT quy đổi sang thang 40 là: {initial_score}\n")
    elif option == 2: print("PT2")
    elif option == 3: print("PT3")
    elif option == 4: print("PT4")
    else: print("Đầu vào không hợp lệ\n")

    conversion_result_table (case_table, option-1, initial_score)

    return

In [119]:
def main_menu():
    print("""Chọn nhóm ngành xét tuyển (
          1: Chương trình Tiêu chuẩn (không bao gồm Chương trình Thương mại) <thang 30>
          2: Chương trình Chất lượng cao và Chương trình tiên tiến (không bao gồm Chương trình Thương mại) <thang 30>
          3: Chương trình Định hướng nghề nghiệp & Phát triển quốc tế <thang 30>
          4: Chương trình Khoa học máy tính & Dữ liệu trong Kinh tế & Kinh doanh <thang 40>
          5: Chương trình tiêu chuẩn Ngôn ngữ thương mại <thang 40>
          6: Chương trình Chất lượng cao Ngôn ngữ thương mại <thang 40> ): """); case = int(input())
    print("\n")
    
    if case == 1: option_1_menu()
    elif case == 2: option_2_menu()
    elif case == 3: option_3_menu()
    elif case == 4: option_4_menu()
    elif case == 5: option_5_menu()
    elif case == 6: option_6_menu()
    else: print("Đầu vào không hợp lệ\n")
    return

# Kết quả

In [120]:
main_menu()

Chọn nhóm ngành xét tuyển (
          1: Chương trình Tiêu chuẩn (không bao gồm Chương trình Thương mại) <thang 30>
          2: Chương trình Chất lượng cao và Chương trình tiên tiến (không bao gồm Chương trình Thương mại) <thang 30>
          3: Chương trình Định hướng nghề nghiệp & Phát triển quốc tế <thang 30>
          4: Chương trình Khoa học máy tính & Dữ liệu trong Kinh tế & Kinh doanh <thang 40>
          5: Chương trình tiêu chuẩn Ngôn ngữ thương mại <thang 40>
          6: Chương trình Chất lượng cao Ngôn ngữ thương mại <thang 40> ): 


Chọn phương thức xét tuyển (
          1: Kết quả THPTQG (khối D) + CCNNQT
          2: Đối tượng HSG Quốc gia + CCNNQT
          3: Đối tượng HSG Tỉnh/Thành phố hoặc hệ Chuyên + CCNNQT
          4: Kết quả DGNL Quôc tế + CCNNQT ): 


Nhập lần lượt các đầu điểm: (nếu bạn thi tổ hợp khác A00, cộng 1 điểm vào môn THPTQG bất kì như Toán/Lý/Hóa/Văn để đổi điểm về A00):
Môn Toán: 
Môn Lý/Hóa/Văn: 
Điểm quy đổi ngoại ngữ: 
Chọn phương thức quy đổi C

ValueError: invalid literal for int() with base 10: ''